In [1]:
import numpy as np
import pandas as pd
import math 
import os
import lvm_read
import pickle
from datetime import datetime
import h5py
import tables

In [2]:
#obtain lvm data
#code from this: https://github.com/openmodal/lvm_read/blob/master/Showcase%20lvm_read.ipynb
os.chdir("C:\\Github Repositories\\LabView-Garmin-To-HDF5")
LVMfilename = 'test_22-10-11_1349_011.lvm'
lvm = lvm_read.read(LVMfilename,read_from_pickle=False)

In [3]:
#create metadata
metaData = {'Channels': lvm[0]['Channels'],
            'Samples': lvm[0]['Samples'],
            'Date': lvm[0]['Date'],
            'Time': lvm[0]['Time'],
            'Y_Unit_Label': lvm[0]['Y_Unit_Label'],
            'X_Dimension': lvm[0]['X_Dimension'],
            'X0': lvm[0]['X0'],
            'Delta_X': lvm[0]['Delta_X']}

In [4]:
#organize file name
#obtain start date
startDate = lvm[0]['Date'][0]
date = startDate.replace('/','-')

#obtain start time
startTime = lvm[0]['Time'][0]

#obtain time values
timeTuple = startTime.split(':',2)

#make file names
fileName = date + '_' + timeTuple[0] + timeTuple[1] + '.hdf5'

In [5]:
#calculate time intervals
deltaX = lvm[0]['Delta_X'][0]
samples = lvm[0]['Samples'][0]

timeFloat = round(float(timeTuple[2]),4)

timeArray = [timeFloat]
count = 1

minute = timeTuple[1]

for i in range (0,samples-1):
    nextTime = timeArray[i] + deltaX
    count = count + 1

    if nextTime >= 60:
        break
    else:
        timeArray.append(nextTime)

In [6]:
#grab Data & convert to pandas dataframe
voltages = []
for i in range (0, count):
    voltage = lvm[0]['data'][i][0]
    voltages.append(voltage)

d = {'Time': timeArray, 'Voltage': voltages}
df = pd.DataFrame(d)

In [7]:
#coding example for HDF5 https://www.geeksforgeeks.org/how-to-add-metadata-to-a-dataframe-or-series-with-pandas-in-python/

#store HDF5 Data
storeData = pd.HDFStore(fileName)
storeData.put('data',df)
storeData.get_storer('data').attrs.metadata = metaData

storeData.close()

with pd.HDFStore(fileName) as storeData:
    data = storeData['data']
    metaData = storeData.get_storer('data').attrs.metadata

In [9]:
#test read
#extra info: https://stackoverflow.com/questions/57146913/unable-to-open-a-previously-opened-h5
f = h5py.File(fileName, 'r')
f.keys()
f.close

pd.read_hdf(fileName,'data')

,Time,Voltage
0,49.6750,8.761779
1,49.6755,8.756186
2,49.6760,8.753224
3,49.6765,8.758160
4,49.6770,8.746314
...,...,...
9995,54.6725,8.749934
9996,54.6730,8.757502
9997,54.6735,8.759476
9998,54.6740,8.754211


In [ ]:
tables.file._open_files.close_all()